Installations

In [1]:
%pip install -U huggingface_hub
%pip install transformers==4.45.2 sentence-transformers==3.1.1
%pip install setfit datasets
%pip install tqdm
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Model training

In [3]:
import pandas as pd
from datasets import Dataset
from setfit import SetFitModel, SetFitTrainer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import shutil

# Load training data
example_df = pd.read_csv("../data/raw/training_dataset.csv")

# Rename the "label" column to "teq_label" (if needed)
example_df = example_df.rename(columns={"label": "teq_label"})

# Convert categorical labels to numbers
label_mapping = {label: idx for idx, label in enumerate(set(example_df["Labels"].tolist()))}
print("Label Mapping (Labels to Numbers):", label_mapping)

example_df["Labels"] = example_df["Labels"].map(label_mapping)

# Split into training and validation sets
train_df, val_df = train_test_split(example_df, test_size=0.2, random_state=1234)
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Load the SetFit model
model = SetFitModel.from_pretrained("sentence-transformers/distiluse-base-multilingual-cased-v2")

# Train the model properly
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Add validation set
    batch_size=16,
    num_epochs=5,
    column_mapping={"filtered_text": "text", "Labels": "label"}
)
trainer.train()
model_save_path = "trained_setfit_model_second_run"
trainer.model.save_pretrained(model_save_path)
# Create a zip archive of the saved model directory
shutil.make_archive(model_save_path, 'zip', model_save_path)

Label Mapping (Labels to Numbers): {'критикует правительство России': 0, 'нейтрален в отношении правительства России': 1, 'выражает поддержку правительству России': 2}


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
<positron-console-cell-6>:29: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
Map: 100%|██████████| 48/48 [00:00<00:00, 3696.58 examples/s]
***** Running training *****
  Num unique pairs = 1920
  Batch size = 16
  Num epochs = 5
  0%|          | 1/600 [00:05<50:56,  5.10s/it]

{'embedding_loss': 0.1994, 'grad_norm': 2.5136122703552246, 'learning_rate': 3.3333333333333335e-07, 'epoch': 0.01}


  1%|          | 6/600 [00:30<50:10,  5.07s/it]

Using the model for classification of the dataset 

In [ ]:
import pandas as pd
from setfit import SetFitModel, SetFitTrainer
from tqdm import tqdm
# Load the dataset for classification
classification_data = pd.read_csv("executive_speeches_trimmed.csv")

model_path = "trained_setfit_model_second_run"
executive_classifier = SetFitModel.from_pretrained(model_path)

# Predict labels using the trained SetFit model
predictions = []
batch_size = 1000  # Adjust batch size as needed

# Use tqdm to show progress
for i in tqdm(range(0, len(classification_data), batch_size), desc="Predicting labels"):
    batch = classification_data["filtered_text"][i:i + batch_size].tolist()
    predictions.extend(executive_classifier.predict(batch))

classification_data["setfit_labels"] = predictions

# Save the updated dataset with predictions
classification_data.to_csv("classified_results_executive_speeches_with_predictions.csv", index=False)